In [1]:
pip list

Package                   Version
------------------------- --------------
accelerate                0.29.3
aiohttp                   3.9.5
aiosignal                 1.3.1
anyio                     4.3.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
async-timeout             4.0.3
attrs                     23.2.0
Babel                     2.14.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.2.2
cffi                      1.16.0
charset-normalizer        3.3.2
click                     8.1.7
comm                      0.2.2
datasets                  2.19.0
debugpy                   1.8.1
decorator                 5.1.1
defusedxml                0.7.1
dill                      0.3.8
evaluate                  0.4.1
exceptiongroup            1.2.1
executing                 2.0.1
fastjsonschema            2.19.1
filelock         

In [2]:
!pip install transformers pandas datasets nltk beautifulsoup4 torch evaluate transformers[torch] scikit-learn

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [3]:
import os 

target_directory = '/export/scratch/khond014-sentiment'

os.chdir(target_directory)

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

nltk_data_path = '/export/scratch/khond014-sentiment'
nltk.data.path.append(nltk_data_path)

nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('stopwords', download_dir=nltk_data_path)


[nltk_data] Downloading package punkt to
[nltk_data]     /export/scratch/khond014-sentiment...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /export/scratch/khond014-sentiment...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, pipeline

from datasets import load_dataset
from bs4 import BeautifulSoup
import re

import numpy as np
import evaluate

In [7]:
metric = evaluate.load("accuracy")

In [8]:
# Load dataset and specify the data directory
dataset = load_dataset('stanfordnlp/imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
def remove_html_tags(example):
    # Extract the 'text' field from the example
    text = example['text']

    # Parse HTML content and remove tags
    soup = BeautifulSoup(text, 'html.parser')
    text_without_html = soup.get_text(separator=' ')
    text_without_html = re.sub(r'\s+', ' ', text_without_html).strip()

    # Update the 'text' field in the example with processed text
    example['text'] = text_without_html

    return example

In [10]:
def clean_review(example):
    # Tokenize the text into words
    tokens = word_tokenize(example['text'])

    # Define stopwords, punctuation characters, and contractions
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    contractions = {
        "n't": "not",
        "'s": "",
        "'re": "are",
        "'ve": "have",
        "'d": "would",
        "'ll": "will",
        "'m": "am"
    }

    # Filter out stopwords, punctuation, and handle contractions
    filtered_tokens = []
    for token in tokens:
        token_lower = token.lower()
        if token_lower in contractions:
            cleaned_token = contractions[token_lower]
            if cleaned_token != "":
                filtered_tokens.append(cleaned_token)
        elif token_lower not in stop_words and token_lower not in punctuation:
            filtered_tokens.append(token)

    # Reconstruct the text from filtered tokens
    cleaned_text = ' '.join(filtered_tokens)

    # Update the 'text' field in the example with cleaned text
    example['text'] = cleaned_text

    return example


In [11]:
def clean_review_text(text):
    # Tokenize the text into words
    tokens = word_tokenize(text)

    # Define stopwords, punctuation characters, and contractions
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    contractions = {
        "n't": "not",
        "'s": "",
        "'re": "are",
        "'ve": "have",
        "'d": "would",
        "'ll": "will",
        "'m": "am"
    }

    # Filter out stopwords, punctuation, and handle contractions
    filtered_tokens = []
    for token in tokens:
        token_lower = token.lower()
        if token_lower in contractions:
            cleaned_token = contractions[token_lower]
            if cleaned_token != "":
                filtered_tokens.append(cleaned_token)
        elif token_lower not in stop_words and token_lower not in punctuation:
            filtered_tokens.append(token)

    # Reconstruct the text from filtered tokens
    cleaned_text = ' '.join(filtered_tokens)

    return cleaned_text

In [12]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [13]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL, num_labels=2, id2label=id2label, label2id=label2id,
    ignore_mismatched_sizes=True
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

In [14]:
def model_tokenizer(examples):
     return tokenizer(examples["text"], padding="max_length", max_length=512, truncation=True)

In [15]:
splits_to_process = ['train', 'test', 'unsupervised']




In [16]:
# Batch size for processing
batch_size=1000

new_dataset = dataset

for split in splits_to_process:
    # Apply the remove_html_tags function to the 'text' column in the specified split
    new_dataset[split] = new_dataset[split].map(remove_html_tags)
    new_dataset[split] = new_dataset[split].map(clean_review)






Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

/tmp/ipykernel_1330160/781736049.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
Parameter 'function'=<function clean_review at 0x7fe191734b80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets = new_dataset.map(model_tokenizer, batched=True, batch_size=batch_size)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [20]:
#  LOOOGITS!!

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
training_args = TrainingArguments(
    output_dir="model_ling_twitter_full",
    learning_rate=2e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.300300,0.362881,0.918280
2,0.224600,0.286519,0.929160
3,0.119800,0.370394,0.930280


TrainOutput(global_step=9375, training_loss=0.2298971268717448, metrics={'train_runtime': 9278.0379, 'train_samples_per_second': 8.084, 'train_steps_per_second': 1.01, 'total_flos': 1.9733329152e+16, 'train_loss': 0.2298971268717448, 'epoch': 3.0})